In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf

AttributeError: module 'tensorflow.core.framework.types_pb2' has no attribute 'SerializedDType'

In [4]:
def skin_segment_image(img):
  hsvim = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  lower = np.array([0, 30, 60], dtype = "uint8")
  upper = np.array([20, 150, 255], dtype = "uint8")
  # lower = np.array([0, 48, 80], dtype = "uint8")
  # upper = np.array([20, 255, 255], dtype = "uint8")
  skinRegionHSV = cv2.inRange(hsvim, lower, upper)
  blurred = cv2.blur(skinRegionHSV, (2,2))
  ret, thresh = cv2.threshold(blurred,0,255,cv2.THRESH_BINARY)
  masked = cv2.bitwise_and(img, img, mask=thresh)
  return masked

def display_output(img, predicted_class):
  _, width, _ = img.shape
  offset = 100
  font = cv2.FONT_HERSHEY_SIMPLEX
  cv2.putText(img, str(predicted_class),
              (width-offset,offset), font, fontScale=3,
              color=(0,0,255), thickness=3)
  cv2.imshow("output", img)
  return img

def save_image(img, pred_class, count, f_name):
  if pred_class is not None:
    cv2.imwrite('out/{}_frame-{:03d}.jpg'.format(pred_class, count), img)
  else:
    cv2.imwrite('{}/frame-{:03d}.jpg'.format(f_name, count), img)
  count += 1
  return count

def define_labels():
  labels = {
      0 : 'aware',
      1 : 'drowsy',
      
    }
  return labels

def recognize():
    labels = define_labels()
    cam = cv2.VideoCapture(0)
    out_count = 1
    thresh_count = 1
    inp_count = 1
    while True:
        img = cam.read()[1]
        img = cv2.flip(img, 1)
        img_copy = img.copy()
        img_resized = cv2.resize(img, (64, 64))
        masked_frame = skin_segment_image(img)
        masked_frame_resized = skin_segment_image(img_resized)
        image_data = tf.keras.utils.img_to_array(masked_frame_resized)
        image_data = np.expand_dims(image_data, axis=0)
        image_data /= 255
        model = tf.keras.models.load_model('model.h5')
        pred = model.predict(image_data)
        predicted_class = labels[np.argmax(pred)]
        ret_img = display_output(img, predicted_class)
        cv2.imshow("thresh", masked_frame)
        if cv2.waitKey(1) == ord('s'):
          inp_count = save_image(img_copy, None, inp_count, 'inp')
          thresh_count = save_image(masked_frame, None, thresh_count, 'thresh')
          out_count = save_image(ret_img, predicted_class, out_count, 'out')
        if cv2.waitKey(1) == ord('q'):
            break
    cam.release()
    cv2.destroyAllWindows

recognize()

AttributeError: 'NoneType' object has no attribute 'copy'